# This is Krita mod to use Stable Diffusion from Collab in Krita editor

got to https://colab.research.google.com/ press Upload and Upload this file

# Instruction:

0. Install [Krita](https://krita.org/en/download/krita-desktop/) (image editor) 

1. Clone/download this to your PC https://github.com/Interpause/auto-sd-krita

2. Install local Stable Diffusion plugin to Krita:

  ### Install plugin

  - Launch Krita.

  - **Find the pykrita folder** - On the ***menubar***, go to ***Settings > Manage Resources...***
  
  - On the popup, ***click Open Resource Folder***. Take note of the **pykrita folder** located inside (or create it if it doesn't exist).

  - **Install the plugin by copying content** of `auto-sd-krita/krita_plugin/` folder to **pykrita folder**. 
  (*copy krita_diff folder and krita_diff.desktop to pykrita*)

  ### Enabling the plugin

  - Restart Krita.

  - On the menubar, go to **Settings > Configure Krita...**

  - On the left sidebar, go to **Python Plugin Manager**.

  - Look for **Stable Diffusion Plugin and tick the checkbox**.

  - Restart Krita again for changes to take effect.
  
  - The SD Plugin docked window should appear on the left of the Krita window. If it does not, look on the menubar under Settings > Dockers for SD Plugin.

3. # Fill **Required** parameters below!

4. Run this Colab script.

5. in Krita in SD Plugin docker window go to **Config** and copy **Backend URL** from this script, look ***section* Copy this address to Krita** below, after Parameters section.


# Warning - **Krita editor** **may** **freeze** during uploading images, so remember to **save** everything **before** pressing "Start" Button in Krita SD Plugin.

If Krita editor freeze - restart it, do not need to restart this page, Krita will connect again.

In [ ]:
#@title Parameters

#@markdown # Tokens and atleast one link is Requied!

#@markdown get a token from https://huggingface.co/settings/tokens
huggingface_user_token = "" #@param {type:"string"}

#@markdown get a token from https://dashboard.ngrok.com/get-started/your-authtoken
ngrok_user_token = "" #@param {type:"string"}

#@markdown #Set atleast one model:

#@markdown model to load
use_model = 'Model 1' #@param ["Model 1", "Model 2", "Model 3", "Model 4"]

#@markdown model URL to download, just to save URL and not type every time
model_1_url = "https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt" #@param {type:"string"}

model_2_url = "https://huggingface.co/hakurei/waifu-diffusion-v1-3/resolve/main/wd-v1-3-float32.ckpt" #@param {type:"string"}

model_3_url = "" #@param {type:"string"}

model_4_url = "" #@param {type:"string"}

#@markdown #WebUI only, look **after launching Actual launch** section to get link

remote_login = "me" #@param {type:"string"}

remote_pass = "test" #@param {type:"string"}

#@markdown #Next happening after manual stoppind WebUI in Collab (by pressing Stop Cell)

#@markdown Save every generated image to your Google drive
save_to_Google_drive_at_end = 'false' #@param ["true", "false"]

#@markdown will show all generated images on this Collab page, after stoping WebUI
show_all_generated_images_at_end = 'false' #@param ["true", "false"]



# Useful links:

https://lexica.art/ - iamges with prompts

https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Features - Features of SD

For better results: (in SD UI)

CFG Scale is "more random or less random to fit your prompt" - 7 is more random, 12 is less, more less

Denoising strength - less is "more close to your original image"(in img2img stuff), 0.45 is closer, 0.75 is more variations

# **Remember to save results** after finishing generation of images! 

---

# Copy this address to Krita SD plugin AFTER **Actual launch** section.
After text - `Copy next URL to Krita SD Backend URL `

Link look like `http://adbd-1234_numbers-...etc.ngrok.io:80` - copy WITH end :80

In [ ]:
# Using https://ngrok.com/ to build tonnel to Collab VM.
!pip install pyngrok
from pyngrok import ngrok

if('ngrok_tunnel' in locals()):
  ngrok.disconnect(ngrok_tunnel.public_url)
ngrok.set_auth_token("2GMrtXqV59us0mvDKFJvZAec39Q_3B4AXzBYSa3gNmxFZT7sv")
ngrok_tunnel = ngrok.connect(8000)
print("\n\n\n Copy next URL to Krita SD Backend URL \n")
print(ngrok_tunnel.public_url+":80")


---


In [ ]:
!uname -a
!nvidia-smi

In [ ]:
import os
import re
models_urls = [model_1_url, model_2_url, model_3_url, model_4_url]
model_index = 0 if use_model == 'Model 1' else 1 if use_model == 'Model 2' else 2 if use_model == 'Model 3' else 3
local_model_url = models_urls[model_index]
local_model_url = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', local_model_url)[0]
if(len(local_model_url)<6):
  raise TypeError("model_url is not correct")
local_file_name = os.path.basename(local_model_url).split('/')[-1]
if(len(local_file_name)<4):
  raise TypeError("file name in model_url is not correct")

Clone webui repository

In [ ]:
if(not os.path.exists("/content/stable-diffusion-webui")):
  !git clone https://github.com/Interpause/auto-sd-krita
  !mv "auto-sd-krita" "stable-diffusion-webui"
%cd stable-diffusion-webui

Download the model from huggingface.

In [ ]:
#@title Model
user_header = f"\"Authorization: Bearer {huggingface_user_token}\""
if(use_model == 'Model 1'):
  tflname = "/content/stable-diffusion-webui/"+local_file_name
  if(not os.path.exists(tflname)):
    !wget --header={user_header} $local_model_url
  if(os.path.exists("/content/stable-diffusion-webui/model.ckpt")):
    !rm "/content/stable-diffusion-webui/model.ckpt"
  !ln -s "$tflname" "/content/stable-diffusion-webui/model.ckpt"

if(not os.path.exists("/content/stable-diffusion-webui/model.ckpt")):
  raise FileNotFoundError("error during creation of model.ckpt")

In [ ]:
!COMMANDLINE_ARGS="--exit" REQS_FILE="requirements.txt" python launch.py

Change into Web UI directory and download updates

In [ ]:
%cd /content/stable-diffusion-webui
!git pull

# **Actual launch** of Krita server.

---

In [ ]:
!COMMANDLINE_ARGS="--share --gradio-debug --gradio-auth $remote_login:$remote_pass" REQS_FILE="requirements.txt" python launch.py

## Commands for ***after*** you have gotten done with a session 

(after stoping previous section)

In [ ]:
ngrok.disconnect(ngrok_tunnel.public_url)

Zip images for downloading on local drive (click the folder icon on the left, the one below {x})

In [ ]:
if(save_to_Google_drive_at_end=="true"):
  # find does rename to fix names to ASCII
  !find /content/stable-diffusion-webui/ -type f -exec bash -c 'for f do d=${f%/*} b=${f##*/} nb=${b//[^A-Za-z0-9._-]/_}; [[ $b = "$nb" ]] || mv "$f" "$d/$nb"; done' _ {} +
  !zip -r /content/stable-diffusion-webui /content/stable-diffusion-webui/outputs 

Save images to Google Drive **Warning: this will cause google to scan your drive, so if you intend to use this and worry about that kind of stuff, probablly just set this up on a clean account that's just for this colab**

In [ ]:
from google.colab import drive

if(save_to_Google_drive_at_end=="true"):
  drive_path = "/content/drive"
  drive.mount(drive_path,force_remount=False)
  !mkdir "/content/drive/MyDrive/waifu-dif/"
  !cp -r "/content/stable-diffusion-webui/outputs" "/content/drive/MyDrive/waifu-dif/"


Display images on this Collab page

In [ ]:
from IPython import display
import glob
folders_to_show = ['img2img-grids', 'img2img-images', 'txt2img-grids', 'txt2img-images', ] # 'krita-in', 'krita-out' # krita temp-folders ignored
ext_to_show = ['png', 'jpg', 'jpeg']
out_path_dir = '/content/stable-diffusion-webui/outputs/'
if(show_all_generated_images_at_end):
  for folder_name in folders_to_show:
    imdir = out_path_dir+folder_name+"/"
    img_files = []
    [img_files.extend(glob.glob(imdir + '*.' + e)) for e in ext_to_show]
    print("Folder "+folder_name+" content:")
    for img_filename in img_files:
      display.Image(img_filename)